In [3]:
import os
from shutil import copyfile

In [4]:
######################################################################
# Prepare dataset for training
# You only need to change this line to your dataset download path
# --------------------------------------------------------------------

download_path = 'D:\Datasets\DukeMTMC-reID'

if 'cuhk' in download_path:
    suffix = 'png'
else:
    suffix = 'jpg'

if not os.path.isdir(download_path):
    print('please change the download_path')

save_path = download_path + '/pytorch'
if not os.path.isdir(save_path):
    os.mkdir(save_path)

In [5]:
query_path = download_path + '/query'
query_save_path = download_path + '/pytorch/query'
if not os.path.isdir(query_save_path):
    os.mkdir(query_save_path)
    
for root, dirs, files in os.walk(query_path, topdown=True):
    for name in files:
        if not name[-3:] == suffix:
            continue
        ID = name.split('_')
        src_path = query_path + '/' + name
        dst_path = query_save_path + '/' + ID[0]
        if not os.path.isdir(dst_path):
            os.mkdir(dst_path)
        copyfile(src_path, dst_path + '/' + name)

In [6]:
# gallery
gallery_path = download_path + '/bounding_box_test'
gallery_save_path = download_path + '/pytorch/gallery'
if not os.path.isdir(gallery_save_path):
    os.mkdir(gallery_save_path)

for root, dirs, files in os.walk(gallery_path, topdown=True):
    for name in files:
        if not name[-3:] == suffix:
            continue
        ID = name.split('_')
        src_path = gallery_path + '/' + name
        dst_path = gallery_save_path + '/' + ID[0]
        if not os.path.isdir(dst_path):
            os.mkdir(dst_path)
        copyfile(src_path, dst_path + '/' + name)

In [7]:
# ---------------------------------------
# train_all
train_path = download_path + '/bounding_box_train'
train_save_path = download_path + '/pytorch/train_all'
if not os.path.isdir(train_save_path):
    os.mkdir(train_save_path)

for root, dirs, files in os.walk(train_path, topdown=True):
    for name in files:
        if not name[-3:] == suffix:
            continue
        ID = name.split('_')
        src_path = train_path + '/' + name
        dst_path = train_save_path + '/' + ID[0]
        if not os.path.isdir(dst_path):
            os.mkdir(dst_path)
        copyfile(src_path, dst_path + '/' + name)

In [31]:
train_path = download_path + '/bounding_box_train'
cnt = 0
root, dirs, files = next(os.walk(train_path, topdown=True))

print(root)

D:\Datasets\DukeMTMC-reID/bounding_box_train


### Official code of Open-Clip

In [ ]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("CLIP.png")).unsqueeze(0)

text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

### For feature extraction

In [35]:
import torch
from PIL import Image
import open_clip

train_path = 'D:/Datasets/DukeMTMC-reID/pytorch/train_all'
print('train_path:', train_path)

feature_path = 'D:/Datasets/DukeMTMC-reID/tensors/train_all'

# get all folders in the train_path
dirs = os.listdir(train_path)
for dir in dirs:
    feature_root= os.path.join(feature_path, dir)
    os.makedirs(feature_root, exist_ok=True)
        
    root, _, files = next(os.walk(train_path + '/' + dir))
    print('root:', root)
    for name in files:
        pre, ext = os.path.splitext(name)
        if ext != suffix:
            continue
        single_image_path = os.path.join(root, name)
        single_image = preprocess(Image.open(single_image_path)).unsqueeze(0)
        with torch.no_grad(), torch.cuda.amp.autocast():
            single_image_features = model.encode_image(single_image)
            single_image_features /= single_image_features.norm(dim=-1, keepdim=True)
            torch.save(single_image_features, os.path.join(feature_root, pre + '.pt'))
    break





train_path: D:/Datasets/DukeMTMC-reID/pytorch/train_all
root: D:\Datasets\DukeMTMC-reID/bounding_box_train
0001_c2_f0046182 .jpg


NameError: name 'preprocess' is not defined

In [36]:
from fast_pytorch_kmeans import KMeans
import torch

kmeans = KMeans(n_clusters=8, mode='euclidean', verbose=1)
x = torch.randn(100000, 64, device='cuda')
labels = kmeans.fit_predict(x)

AssertionError: Torch not compiled with CUDA enabled

In [3]:
import os.path as osp
import numpy as np
import torch
import glob
import re

In [10]:
import os.path as osp
import numpy as np
import torch
import glob
import re

dir_path = "/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all"
tensor_paths = glob.glob(osp.join(dir_path, '*/*.pt'))


In [5]:
print(tensor_paths[:10])
print(len(tensor_paths))



['/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c6_f0110693.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c8_f0088384.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c7_f0115296.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c6_f0110813.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c8_f0088264.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c6_f0111053.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c6_f0111173.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c7_f0115056.pt', '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/DukeMTMC-reID/tensors/train_all/4509/4509_c6_f0110933.pt', '/home/zh

In [7]:
# 将路径下所有的tensor保存成一个全部tensor，维度为（数量，维度）
all_tensors = []
for tensor_path in tensor_paths:
    tensor = torch.load(tensor_path)
    all_tensors.append(tensor)
all_tensors = torch.cat(all_tensors, dim=0)
all_tensors = all_tensors.cuda()


torch.Size([243, 512])


In [11]:
from fast_pytorch_kmeans import KMeans
# 使用kmeans聚类
k = 25
sim_mode = 'euclidean'
kmeans = KMeans(n_clusters=k, mode=sim_mode, verbose=1)
labels = kmeans.fit_predict(all_tensors)
centroids = kmeans.centroids

path2label = {}
for i, tensor_path in enumerate(tensor_paths):
    file_name = osp.basename(tensor_path)
    file_name = re.sub(r'\.pt$', '', file_name)
    label = labels[i].item()
    path2label[file_name] = label

print(labels.shape)
print(centroids.shape)
# 保存labels和centroids
result={}
result['labels'] = labels
result['path2label'] = path2label 
result['centroids'] = centroids # shape=(num_clusters, feature_dim)
print(result)


# save_path = "/Kmeans_result/DukeMTMC-reID/{}k_{}.pt".format(k, sim_mode)
# torch.save(result, save_path)



used 6 iterations (0.0288s) to cluster 243 items into 25 clusters
torch.Size([243])
torch.Size([25, 512])
{'labels': tensor([ 0,  8,  8,  8,  8,  8,  0,  8,  0, 19,  0,  1, 19,  8,  9,  1,  9,  9,
        16, 23,  2, 24, 24,  8,  1,  9, 23,  5,  9,  2,  1,  2,  5, 18,  5,  2,
        21, 21,  5,  1, 10,  5, 12, 24, 19, 12,  5,  5,  1, 19, 19,  5, 10,  3,
         5, 24,  3,  5, 24,  5,  5, 19, 12,  3, 12,  3,  3, 24,  5, 13, 24,  5,
         5,  5, 17,  5,  5,  5,  5,  5, 17,  5, 23, 13, 11,  5, 13, 11,  8,  5,
         8,  8,  8,  8,  0,  5,  0, 15, 14, 19,  0, 13, 13,  8,  8, 21, 15, 13,
         9,  5,  6, 23, 19, 13,  8,  5,  5,  5,  5, 13,  8,  5,  8, 13, 13,  8,
        17,  5, 16, 23,  8,  5,  8,  0,  8,  5, 18,  9,  9, 21,  8, 19, 21,  9,
        23, 18, 21,  9,  5, 17,  5, 11,  8,  9, 21,  9, 21,  9,  7,  8, 13, 21,
         8,  4,  8,  3,  7,  4,  5,  5,  4,  4,  7,  4,  5,  1,  7,  7,  7,  4,
         5,  4,  4,  7,  1,  4,  7,  9,  8, 20,  1,  1, 20,  8,  8, 23,  1, 17,
   

In [10]:
print(img_paths[:10])
pattern = re.compile(r'([-\d]+)_c(\d)')
pid, cam = map(int, pattern.search(img_paths[0]).groups())
print(pid, cam)


['D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046182.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046302.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046422.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046542.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046662.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046782.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0046902.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c2_f0047022.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c5_f0051247.jpg', 'D:/Datasets/DukeMTMC-reID/pytorch/train_all\\0001\\0001_c5_f0051367.jpg']
1 2
